In [1]:
import obspy 
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
from stdb import load_db 
import pandas as pd
import pickle
import glob, os
import pygmt
from obspy.core.event import Catalog
from obspy.core.event.catalog import read_events
from obspy.clients.fdsn import Client
import warnings  
warnings.filterwarnings("ignore")  
client = Client('IRIS')

In [65]:
phase = 'SKKS'
DATApath = f'/Volumes/home/Research/STEP/01_Split/DATA_{phase}'
eventtime = ['20131123_074832']
CSVpath = f'/Volumes/home/Research/STEP/02_Station_result_csv/*Mw6*{phase}'
SAVEpath = '/Volumes/home/Research/STEP/03_Split_result_eventmap'

In [63]:
def Calc_rho(RCdt, SCdt):
    rho = RCdt/SCdt 
    return rho
def Calc_Phi(RCPhi, SCPhi):
    Phi = max (abs(RCPhi-SCPhi), abs(SCPhi-RCPhi))
    if Phi > 90: Phi = 180 - Phi
    return Phi
def ReadPKL_manual(evtpath):
    Meta_pkl = evtpath + '/Meta_manual.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Split_result_pkl = evtpath + '/Split_results_manual.pkl'
    SC_result_pkl = evtpath + '/SC_res_manual.pkl'
    RC_result_pkl = evtpath + '/RC_res_manual.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    Split_result = pickle.load(open(Split_result_pkl, "rb"))
    SC_res = pickle.load(open(SC_result_pkl, "rb"))
    RC_res = pickle.load(open(RC_result_pkl, "rb"))
    return Meta, Station_data, Split_result, SC_res, RC_res
def ReadPKL_auto(evtpath):
    Meta_pkl = evtpath + '/Meta_data.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    return Meta, Station_data
def PygmtBegin(figmap, region):
    pygmt.config(FONT_ANNOT_PRIMARY = '10p,Times-Roman',
                FONT_LABEL = '8',
                MAP_LABEL_OFFSET = '4p',
                MAP_TICK_LENGTH_PRIMARY = '5p',
    #             PS_MEDIA ='A4',
                PROJ_LENGTH_UNIT = 'c',
                MAP_ANNOT_OFFSET_PRIMARY = '5p',
                MAP_ANNOT_OFFSET_SECONDARY = '5p',
                MAP_FRAME_TYPE = 'fancy',
                MAP_FRAME_WIDTH = 'auto',
                FORMAT_GEO_MAP = 'D',
                PS_CHAR_ENCODING = 'Standard+',
                MAP_FRAME_PEN = '2p,black')
#     grid = pygmt.datasets.load_earth_relief(resolution="30s", region=region)
    pygmt.makecpt(cmap="elevation", series=[0, 4000, 500],output = 'eleva.cpt', continuous=True)
    figmap.basemap(region=region, projection="M15c", frame = ['x1f0.5', 'y1f0.5', 'nSeW'])
    figmap.grdimage('@earth_relief_30s',region =region, cmap="eleva.cpt",monochrome=True, shading=True, transparency=50)
    figmap.coast(resolution = 'h', shorelines ='1/thinnest,black', water='white', borders = '1/0.25p')
##==========legned setting 
    figmap.plot(x=40.5,y=39.1,style="c0.5c", pen="black", no_clip=True)
    figmap.text(x=40.5,y=38.9,text='Null', font="12p,Times-Bold,black", no_clip=True)
    RCdata = [[41.5, 39.1, 0, 5, 1*40]]
    SCdata = [[41.5, 38.9, 0, 5, 1*40]]
    figmap.plot(data=RCdata, style="J", color="red3", pen="0.01p,black", no_clip=True)
    figmap.text(x=42,y=39.1,text='RC', font="12p,Times-Bold,black", no_clip=True)
    figmap.plot(data=SCdata, style="J", color="deepskyblue1", pen="0.01p,black", no_clip=True)
    figmap.text(x=42,y=38.9,text='SC', font="12p,Times-Bold,black", no_clip=True)
    legenddata=[[43, 39.1, 0, 5, 0.5*40],[43.8, 39.1, 0, 5, 1*40],[44.8, 39.1, 0, 5, 1.5*40]]
    figmap.plot(data=legenddata, style="J", color="black", pen="0.01p,black", no_clip=True)
    figmap.text(x=43,y=38.9,text='0.5s', font="12p,Times-Bold,black", no_clip=True)
    figmap.text(x=43.8,y=38.9,text='1s', font="12p,Times-Bold,black", no_clip=True)
    figmap.text(x=44.8,y=38.9,text='1.5s', font="12p,Times-Bold,black", no_clip=True)

In [ ]:
regvals = [40, 47, 39.5, 44.5]
for evt in eventtime:
    figmap = pygmt.Figure()
    PygmtBegin(figmap, regvals)
    
    print(evt)
    for STApath in sorted(glob.glob(f'{DATApath}*ZRT/*/{evt}')):
        # print(STApath)
        NETSTA=STApath.rsplit('/')[-2]

        # figmap.text(x=40, y=45, text =  f'{evt} Phase:{phase}', font='15p',justify="LB", no_clip=True)
        for path in glob.glob(f'{CSVpath}/{NETSTA}*.csv'):
            df = pd.read_csv(path)
            evtdf = df[df['Event'] == evt]
            if len(evtdf) == 0 :
               continue
            elif len(evtdf) != 0 :
                try:
                    Meta, Station_data, Split_result, SC_res, RC_res = ReadPKL_manual(STApath)
                    st_lat = Station_data.latitude 
                    st_lon = Station_data.longitude
                    Picktype = 'manual'
                    print(Picktype)
                    CpH  = Meta.CpH
                    RCPhi = RC_res.phi
                    RCdt = RC_res.dtt
                    SCPhi = SC_res.phi
                    SCdt = SC_res.dtt

                    rho = Calc_rho(RCdt, SCdt)
                    phi = Calc_Phi(RCPhi, SCPhi)

                    if 25 < phi < 68 or CpH > 0.76:
                        print('Quality Null')
                        figmap.plot(x=st_lon, y=st_lat, style="c0.15c",color='black', pen="white")
                        figmap.plot(x=st_lon,y=st_lat, style="c0.3c", pen="1p,black")
                    elif 0.8 < rho < 1.1 and phi < 8:
                        t=0
                        print('Quality Good')
                        RCdata = [[st_lon, st_lat, RCPhi-90, 6, RCdt*40]]
                        figmap.plot(data=RCdata, style="J", color="deepskyblue1", pen="0.1p,black",transparency=t)   
                        SCdata = [[st_lon, st_lat, SCPhi-90, 6, SCdt*40]]
                        figmap.plot(data=SCdata, style="J", color="red3", pen="0.1p,black",transparency=t) 
                        figmap.plot(x=st_lon, y=st_lat,style="c0.25c",color='white' ,pen="0.8p,black")
                    elif 0.7 <= rho < 1.2 and phi <= 25:
                        t=0
                        print('Quality Fair')
                        RCdata = [[st_lon, st_lat, RCPhi-90, 6, RCdt*40]]
                        figmap.plot(data=RCdata, style="J", color="deepskyblue1", pen="0.1p,black",transparency=t)   
                        SCdata = [[st_lon, st_lat, SCPhi-90, 6, SCdt*40]]
                        figmap.plot(data=SCdata, style="J", color="red3", pen="0.1p,black",transparency=t) 
                        figmap.plot(x=st_lon, y=st_lat,style="c0.25c",color='white' ,pen="0.8p,black")
                    else:
                        t=50
                        print('Quality Poor')
                        RCdata = [[st_lon, st_lat, RCPhi-90, 6, RCdt*40]]
                        figmap.plot(data=RCdata, style="J", color="deepskyblue1", pen="0.1p,black",transparency=t)   
                        SCdata = [[st_lon, st_lat, SCPhi-90, 6, SCdt*40]]
                        figmap.plot(data=SCdata, style="J", color="red3", pen="0.1p,black",transparency=t) 
                        figmap.plot(x=st_lon, y=st_lat,style="c0.25c",color='white' ,pen="0.8p,black")

                except:
                    Meta, Station_data = ReadPKL_auto(STApath)
                    st_lat = Station_data.latitude 
                    st_lon = Station_data.longitude
                    Picktype = 'auto'
                    print(Picktype)
                    figmap.plot(x=st_lon, y=st_lat, style="c0.15c",color='black', pen="white")
            figmap.text(x=st_lon, y=st_lat-0.15, text=Station_data.station, font="8p,Times-Bold,black",fill='white',transparency=30) 
    SAVEfile = f'{SAVEpath}/{evt}_{phase}'
    figmap.show()
    figmap.savefig(f'{SAVEfile}.png', dpi=200)
    figmap.savefig(f'{SAVEfile}.pdf')
